# Alpha, local grid rotation

This script calculates the local grid rotation needed to transform wind direction in cardinal directions.
* This works for MEPS and AROME-Arctic datasets
more information: [https://github.com/metno/NWPdocs/wiki/Examples#extracting-and-plotting-a-variable]

Calculates "Alpha" at every grid in inputfile and writes as seperate output file (NC-file).

Time: ?? min and Filesize: ?? KB

In [ ]:
#!pip3 install geopy

# Both the functions for distance calculates the harversine distance
# The first function use built in module, not showing the process

import geopy.distance

# define a function calculating the Harversine distance

def distance(origin, destination):
    """
    (Source: https://stackoverflow.com/questions/19412462/
            getting-distance-between-two-points-based-on-latitude-longitude)
    """
    
    d = geopy.distance.geodesic(origin, destination).km

    return d

In [ ]:
import xarray as xr
import numpy as np
import netCDF4 as nc
"""
This routine calculates the local grid rotation (alpha) from input file, 
and writes to a separate output file.
Formula:
  alpha = atan2(dlatykm,dlonykm)*180/pi - 90)

Wind direction relative to Earth (wdir) may later be calculated as follows:
   wdir = alpha + 90-atan2(v, u)
where u(x) and v(y) are model wind relative to model grid

"""


# Function calculating Harversine distance, more elaborate.

def distance(origin, destination):
    """
    (Source: https://stackoverflow.com/questions/19412462/
             getting-distance-between-two-points-based-on-latitude-longitude)

    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    >>> origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (np.sin(dlat / 2) * np.sin(dlat / 2) +
         np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) *
         np.sin(dlon / 2) * np.sin(dlon / 2))
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = radius * c

    return d


In [ ]:
import xarray as xr
import numpy as np
import netCDF4 as nc

# use a random arome arctic file 
infile = "https://thredds.met.no/thredds/dodsC/aromearcticarchive/"+\
         "2022/01/01/arome_arctic_full_2_5km_20220101T00Z.nc"


nc = xr.open_dataset(infile)       

# Variables 
xx = nc.x
yy = nc.y
lat = nc.latitude
lon = nc.longitude
alpha=lat #Target matrix


for j in range(0,yy.size-1):
    for i in range(0,xx.size-1):
        # Prevent out of bounds
        if j==yy.size-1:
            j1=j-1; j2=j
        else:
            j1=j; j2=j+1
        if i==xx.size-1:
            i1=i-1; i2=i
        else:
            i1=i; i2=i+1

        dlatykm=distance([lat[j1,i1],lon[j1,i1]],[lat[j2,i1],lon[j1,i1]])
        dlonykm=distance([lat[j1,i1],lon[j1,i1]],[lat[j1,i1],lon[j2,i1]])

        alpha[j,i]=np.arctan2(dlatykm,dlonykm)*180/np.pi - 90
        


In [ ]:
import netCDF4 as nc

# Make NetCDF file

outfile = "alpha.nc"

rg = nc.Dataset(outfile, 'w', format='NETCDF4')
x=rg.createDimension("x",xx.size)
y=rg.createDimension("y",yy.size)
alph=rg.createVariable("alpha","f4",("y","x"))
alph[:]=alpha
rg.close()